# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [12]:
# 1) Seu código aqui

url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
data = requests.get(url).json()
df = pd.DataFrame(data['items'])

In [15]:
rj_df = df[df['uf'] == 'RJ']

In [17]:
# 2) Seu código aqui
status_counts = df['status'].value_counts()
top_status = status_counts.head(3)
print(top_status)

Deferido             2720
Deferido (PVL-IF)    1009
Arquivado             428
Name: status, dtype: int64


In [36]:
# 3) Seu código aqui
df['ano_status'] = pd.to_datetime(df['status']).dt.year


In [37]:
# 4
year_counts = df['ano_status'].value_counts()
print(year_counts)

1970    5000
Name: ano_status, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [38]:
#1) Seu código aqui
def get_pvl_data(uf, tipo_de_interessado):
    url = f"https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?sg_uf_interessado={uf}&tp_interessado={tipo_de_interessado}"
    data = requests.get(url).json()
    df = pd.DataFrame(data['items'])
    return df

In [39]:
# 2) Seu código aqui
mg_df = get_pvl_data('MG', 'Estado')
num_arquivado = len(mg_df[mg_df['status'] == 'Arquivado por decurso de prazo'])
num_arquivado

264

In [40]:
# 3) Seu código aqui
ba_df = get_pvl_data('BA', 'Município')
ba_deferido = ba_df[ba_df['status'] == 'Deferido']
top_deferido = ba_deferido['interessado'].value_counts().idxmax()
top_deferido

'Cascavel'

In [41]:
# 4) Seu código aqui
ba_df = get_pvl_data('BA', 'Estado')
ba_df.to_csv('bahia_estado.csv', index=False)